# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   393.36   2.033841e-01 -1.361220e+00 -1.157836e+00 ||    319.99   1.654485e-01 1.459675e+00 1.625124e+00 || 
     2   381.67   1.973404e-01 -1.355142e+00 -1.157802e+00 ||    301.65   1.559663e-01 1.469200e+00 1.625167e+00 || 
     3   360.72   1.865062e-01 -1.344250e+00 -1.157744e+00 ||    318.20   1.645234e-01 1.460538e+00 1.625062e+00 || 
     4   332.03   1.716747e-01 -1.329345e+00 -1.157670e+00 ||    366.39   1.894374e-01 1.435375e+00 1.624813e+00 || 
     5   297.64   1.538908e-01 -1.311482e+00 -1.157591e+00 ||    438.58   2.267652e-01 1.397668e+00 1.624434e+00 || 
     6   259.77   1.343106e-01 -1.291825e+00 -1.157515e+00 ||    523.29   2.705611e-01 1.353390e+00 1.623951e+00 || 
     7   220.60   1.140614e-01 -1.271511e+00 -1.157449e+00 ||    606.05   3.133538e-01 1.310056e+00 1.623410e+00 || 
     8   182.06   9.413202e-02 -1.251530e+00 -1.157398e+00 ||    670.97   3.469159e-01 1.27596

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   240.11   1.241467e-01 -6.928317e-02 5.486350e-02 ||    476.40   2.463173e-01 2.136498e+00 2.382816e+00 || 
     2   250.99   1.297693e-01 -7.490581e-02 5.486350e-02 ||    569.72   2.945682e-01 2.088248e+00 2.382816e+00 || 
     3   319.08   1.649793e-01 -1.101158e-01 5.486350e-02 ||    728.56   3.766956e-01 2.006120e+00 2.382816e+00 || 
     4   443.95   2.295414e-01 -1.746779e-01 5.486350e-02 ||    939.59   4.858081e-01 1.897008e+00 2.382816e+00 || 
     5   622.87   3.220491e-01 -2.671856e-01 5.486350e-02 ||   1184.70   6.125354e-01 1.770280e+00 2.382816e+00 || 
     6   850.23   4.396035e-01 -3.847400e-01 5.486350e-02 ||   1441.09   7.451030e-01 1.637713e+00 2.382816e+00 || 
     7  1116.94   5.775045e-01 -5.226410e-01 5.486350e-02 ||   1681.53   8.694195e-01 1.513396e+00 2.382816e+00 || 
     8  1409.94   7.289940e-01 -6.741305e-01 5.486350e-02 ||   1874.62   9.692545e-01 1.413561e+00 2.

## Scale velocities to control temperature

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -1.636277e-01 4.318834e-02 ||    400.00   2.068160e-01 2.166748e+00 2.373564e+00 || 
     2   400.00   2.068160e-01 -2.514156e-01 -4.459962e-02 ||    400.00   2.068160e-01 2.142942e+00 2.349758e+00 || 
     3   400.00   2.068160e-01 -3.481554e-01 -1.413394e-01 ||    400.00   2.068160e-01 2.079058e+00 2.285874e+00 || 
     4   400.00   2.068160e-01 -4.495082e-01 -2.426922e-01 ||    400.00   2.068160e-01 1.985463e+00 2.192279e+00 || 
     5   400.00   2.068160e-01 -5.520308e-01 -3.452148e-01 ||    400.00   2.068160e-01 1.874787e+00 2.081603e+00 || 
     6   400.00   2.068160e-01 -6.530718e-01 -4.462559e-01 ||    400.00   2.068160e-01 1.757918e+00 1.964734e+00 || 
     7   400.00   2.068160e-01 -7.506231e-01 -5.438071e-01 ||    400.00   2.068160e-01 1.642859e+00 1.849675e+00 || 
     8   400.00   2.068160e-01 -8.431754e-01 -6.363594e-01 ||    400.00   2.068160e-01 1.535086

## Langevin Thermostat

In [6]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   743.70   3.845204e-01 -1.344124e+00 -9.596039e-01 ||    186.60   9.647746e-02 1.415712e+00 1.512189e+00 || 
     2   695.07   3.593780e-01 -1.300138e+00 -9.407605e-01 ||    247.55   1.279921e-01 1.381929e+00 1.509921e+00 || 
     3   586.50   3.032435e-01 -1.234606e+00 -9.313623e-01 ||    334.87   1.731437e-01 1.338521e+00 1.511664e+00 || 
     4   436.52   2.256978e-01 -1.157473e+00 -9.317751e-01 ||    426.11   2.203164e-01 1.287397e+00 1.507713e+00 || 
     5   306.77   1.586099e-01 -1.084089e+00 -9.254788e-01 ||    504.22   2.607029e-01 1.238967e+00 1.499670e+00 || 
     6   208.50   1.078050e-01 -1.024848e+00 -9.170434e-01 ||    570.50   2.949718e-01 1.200945e+00 1.495917e+00 || 
     7   144.57   7.474816e-02 -9.931860e-01 -9.184378e-01 ||    609.14   3.149498e-01 1.182149e+00 1.497099e+00 || 
     8   144.94   7.493907e-02 -9.892812e-01 -9.143422e-01 ||    597.99   3.091853e-01 1.18675